# Support Vector Machines



Where SVMs become really powerful is when it is combined with so called _kernels_. This kernels allow us to find linear decision boundaries in higher dimensions when in the original dimension no linear boundary can be found.

We will not discussed exactly on how the Support Vector Machine is computed, because the details become somewhat technical. However, it turns out that the solution involves only the _inner products_ (_dot products_) of the observations as opposed to the observations themselves. The inner product of two observations $x_i$ and $x_{i'}$ is given by

$$
\langle x_i,x_{i'} \rangle = \sum^p_{j=1}x_{ij}x_{i'j}
$$

which in form of the linear classifier $f(x)$ can be represented as

$$
f(x) = \beta_0 + \sum^n_{i=1}\alpha_i \langle x,x_i \rangle
$$

where there are n parameters $\alpha_i$, $i = 1,...,n$, one per training observation.

To estimate the parameters $\alpha_1,...,\alpha_n$ and $\beta_0$, all we need are the $\binom{n}{2}$ inner products $\langle x_i,x_{i'} \rangle$ between all pairs of training observations.

In order to evaluate the function $f(x)$, we need to calculate the inner product between the new point $x$ and each of the training points $x_i$. However it turns out that $\alpha_i$ is nonzero for the support vectors only. If the training observation is not a support vector, $\alpha_i$ turns exactly zero. To sum up, we define a collection $S$ with indices of these support points, therefore we can rewrite any solution function of the form as

$$
f(x) = \beta_0 + \sum_{i \in S}\alpha_i \langle x,x_i \rangle
$$

which obviously contains much fewer terms than all the training observations.

To demonstrate how kernels work, we begin with the imports we need to provide some examples and define our function to plot the decision boundaries:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits import mplot3d

from scipy import stats
from sklearn.datasets import make_blobs, make_circles
from sklearn.svm import SVC

plt.style.use('ggplot')
sns.set()

In [ ]:
sns.set()
plt.rcParams['figure.figsize'] = (10.0, 5.0)
plt.rcParams['image.cmap'] = 'Dark2'

## Linear Kernel

Now suppose that every time the inner product appears in the calculation of the solution for the support vector classifier, we can replace it with a generalization of the form

$$
K(x_i,x_{i'})
$$

where $K$ is some function we refer to as _kernel_. We already discussed the linear kernel which simply put gives as back the Support Vector Classifier

$$
K(x_i,x_{i'}) = \beta_0 + \sum^p_{j=1}x_{ij}x_{i'j}
$$

The linear kernel essentially quantifies the similarity between two observations using Pearson (standard) correlation. The linear kernel describes a linear relationship between to observations to quantify if a obersavtion is more similar to one or the other class.

In [ ]:
def plot_svc_decision_function(model, ax = None, plot_support = True):
    if ax is None:
        ax = plt.gca()
        
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    x = np.linspace(xlim[0], xlim[1], 30)
    y = np.linspace(ylim[0], ylim[1], 30)
    Y, X = np.meshgrid(y, x)
    xy = np.vstack([X.ravel(), Y.ravel()]).T
    P = model.decision_function(xy).reshape(X.shape)

    ax.contour(X, Y, P, colors = 'k', levels = [-1, 0, 1], alpha = 0.5, linestyles=['--', '-', '--'])
    
    if plot_support:
        ax.scatter(model.support_vectors_[:, 0], model.support_vectors_[:, 1], s = 300, lw = 1, facecolors = 'red', alpha = 0.2);
        
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

X, y = make_blobs(n_samples = 30, centers = 2, random_state = 0, cluster_std = 0.6)

model = SVC(kernel = 'linear', C = 1E10)
model.fit(X, y)

plt.scatter(X[:, 0], X[:, 1], c = y, s = 40)
plot_svc_decision_function(model);

## Polynomial Kernel

More often than not, our data is not separatable with a linear decision boundary.<br>
Let's take a look at the following data set which consists observations of drug usages to cure a sickness.

The dosage of the drug determines if the patient got cured (green) or is still sick (black). After taking a look at the observations we can clearly see that if the taken dosage is too little or too much the disease was not cured.

In [ ]:
df = pd.read_csv('drug_dosages.csv')

In [ ]:
df.sample(n = 5)

In [ ]:
X = df['dosage'].values
y = np.zeros(X.shape)
colors = np.where(df['status'] == 'sick', 1, 0)

In [ ]:
plt.figure(figsize = (7, 2))

plt.xlim(0, 29)
plt.ylim(-0.5, 0.5)
plt.yticks([])

plt.scatter(X, y, c = colors, s = 40)
plt.xlabel('\ndosage (mg)')
plt.show()

This introduces a problem because this classes are not linear separatable, but what if we transform this data points to another dimension?

This trick would help us to draw a linear decision boundary on a higher dimension. Consider a transformation of the data into the two-dimensional space where we can compare the similarities of the observations using the polynomial kernel

$$
K(x_i,x_{i'}) = (\beta_0 + \sum^p_{j=1}x_{ij}x_{i'j})^d
$$

with a degree of $d$, where $d$ is a positive integer. Using this kernel leads to a much more flexible decision boundary by fitting a Support Vector Classifier in a higher dimensional space involving polynomials rather than in the original feature space.

When the Support Vector Classifier is combined with a non-linear kernel such as the polinomial kernel, the resulting classifier is known as the Support Vector Machine. In this example, the (non-linear) function has the form of

$$
f(x) = \beta_0 + \sum_{i \in S} \alpha_i K(x, x_i)
$$

where $\alpha_i$ defines the slope and $\beta_0$ the constant term.

To demonstrate, we choose a degree of $d = 2$, a slope $\alpha = 1$ and a coefficient of $\beta_0 = 1$ and now quantify the similarity of two observations $x_i$ and $x_{i'}$. Our polynomial kernel function is now defined as

$$
K(x_i,x_{i'}) = (1 + \sum^p_{j=1}x_{ij}x_{i'j})^2
$$

To simplify the process, let's quantify the similarity of two observations $x_{i1} = a$ and $x_{i'1} = b$, which inserted in our kernel look like the following equation

\begin{align}
K(a,b) &= (1 + a \times b)^2\\
&= (1 + a \times b)(1 + a \times b)\\
&= 2ab + a^2b^2 + 1\\
&= (\sqrt{2}a, a^2, 1) \cdot (\sqrt{2}b, b^2, 1)
\end{align}

This dot product describes our data points in the two-dimensional space, where the first term describes the x-value, the second term the y-value and the third-term the z-value. Therefore our new x-values move with a factor of $\sqrt{2}$ down the x-axis, the y-value is simply the original x-value quared ($x^2$) and Our z-value is a constant term of one.

With this trick, the calculated dot product describes our data points in all dimensions without the need to transform the data into other dimensions. We refer to this trick as _kernel trick_.

To demonstrate, our transformed data points can be displayed on a two-dimensional space looking like the following:

In [ ]:
X = X * np.sqrt(2)
y = X ** 2

plt.scatter(X, y, c = colors, s = 40)
plt.xlabel('\ndosage (mg)')
plt.ylabel('\nsquared dosage (mg)')
plt.show();

To demonstrate, we can now fit a SVM again to show the (linear) decision boundary, which will be a straight line.

Please note two things:
* The SVM does NOT transform data into other dimensions, it simply calculates the similaritie between observation points using higher dimensions
* The SVM will NOT fit a linear kernel after the transformation, so the following graph is only for demonstration purposes

In [ ]:
a = np.reshape(X, (-1, 1))
b = np.reshape(y, (-1, 1))
c = np.append(a, b, axis = 1)

model = SVC(kernel = 'linear', C = 1E10).fit(c, colors)

plt.scatter(a[:, 0], b[:, 0], c = colors, s = 40)
plot_svc_decision_function(model);

We now know the SVM calculates the similarities between observations and therefor can draw a polynomial decision boundary to separate between the two classes.

When we now fit a SVM using a polynomial kernel, the decision boundary looks like the following:

In [ ]:
X = df['dosage'].values
X = np.append(np.reshape(X, (-1, 1)), np.reshape(np.ones(X.shape), (-1, 1)), axis = 1)

fig, ax = plt.subplots(1, 2, figsize = (14, 5))
fig.subplots_adjust(left = 0.0625, right = 0.95, wspace = 0.1)

for axi, C in zip(ax, [1E-1, 1E2]):
    model = SVC(kernel = 'poly', degree = 2, C = C, gamma = 1).fit(X, colors)
    axi.set_xlim(0, 29)
    axi.set_ylim(0, 2)
    y_axis = axi.get_yaxis()
    y_axis.set_visible(False)
    axi.scatter(X[:, 0], X[:, 1], c = colors, s = 40)
    plot_svc_decision_function(model, axi)
    axi.set_title('C = {0:.1f}'.format(C), size = 14)

## Radial Kernel (Gaussian RBF Kernel)

The polynomial kernel as shown above is one example of a vast number of non-linear kernels. Another popular choice is the _radial kernel_ using the radial basis function, which takes the form of

$$
K(x_i,x_{i'}) = exp(-\gamma\sum^p_{j=1}(x_{ij}-x_{i'j})^2)
$$

where $\gamma$ is a positive constant.

Alternatively the radial kernel could be implemented using

$$
K(x_i,x_{i'}) = exp(-\frac{\sum^p_{j=1}(x_{ij}-x_{i'j})^2}{2\sigma^2})
$$

with $\sigma$ as a tuning parameter, which plays a major role in the performance of the kernel. If overestimated, the exponential will behave almost linearly and the higher-dimension projection will start to loose its non-linear nature. On the other hand, if $\sigma$ gets unterestimated, the function will lack regularization and the decision boundary will be highly sensitive to noise in the training data.

Now let's take a look at our next data set, where the radial kernel would be the best fit:

In [ ]:
X, y = make_circles(100, factor = .1, noise = .1)
plt.scatter(X[:, 0], X[:, 1], c = y, s = 40);

To show, how a transformation to other dimensions might look like and to simplify the process again, we define $\gamma = \frac{1}{2}$ and quantify the similarity of two observations $x_{i1} = a$ and $x_{i'1} = b$, which put in our kernel look like the following equation:

\begin{align}
K(a,b) &= e^{-\frac{1}{2}(a-b)^2}\\
&= e^{-\frac{1}{2}(a^2+b^2-2ab)}\\
&= e^{-\frac{1}{2}(a^2+b^2)}e^{-\frac{1}{2}-2ab}\\
&= e^{-\frac{1}{2}(a^2+b^2)}e^{ab}
\end{align}

To now proceed, we can make use of the Taylor Series Expansion

$$
\sum^{\infty}_{n=0}\frac{f^{(n)}(a)}{n!}(x-a)^n
$$

and expand the last term of our kernel

$$
e^{ab} = 1+\frac{1}{1!}ab+\frac{1}{2!}(ab)^2+\frac{1}{3!}(ab)^3+...+\frac{1}{\infty!}(ab)^{\infty}
$$

To know find the inner product which describe the position of our observations again, let's revisit the polynomial kernel again and think of the inner product for an infinite number of dimensions:

$$
a^0b^0+a^1b^1+a^2b^2+...+a^{\infty}b^{\infty} = (1, a, a^2, ..., a^{\infty})\cdot(1, b, b^2, ..., b^{\infty})
$$

One thing catches our eye immediately, because a polynomial kernel with
* $\beta_0 = 0$ and $d = 0$ equals $1$
* $\beta_0 = 0$ and $d = 1$ equals $ab$
* $\beta_0 = 0$ and $d = 2$ equals $(ab)^2$
* ...
* $\beta_0 = 0$ and $d = \infty$ equals $(ab)^{\infty}$

We can make use of this and with that in mind, we can easily find the dot product of $e^{ab}$ in our radial kernel:

$$
(1, \sqrt{\frac{1}{1!}a}, \sqrt{\frac{1}{2!}a^2}, \sqrt{\frac{1}{3!}a^3}, ..., \sqrt{\frac{1}{\infty!}a^{\infty}})
\cdot
(1, \sqrt{\frac{1}{1!}b}, \sqrt{\frac{1}{2!}b^2}, \sqrt{\frac{1}{3!}b^3}, ..., \sqrt{\frac{1}{\infty!}b^{\infty}})
$$

Going back to our original radial kernel function, we can now plug in the inner product of $e^{ab}$ and our function is now equal to this term

\begin{align}
K(a,b) &= e^{-\frac{1}{2}(a-b)^2}\\
&= e^{-\frac{1}{2}(a^2+b^2)}
[
(1, \sqrt{\frac{1}{1!}a}, \sqrt{\frac{1}{2!}a^2}, \sqrt{\frac{1}{3!}a^3}, ..., \sqrt{\frac{1}{\infty!}a^{\infty}})
\cdot
(1, \sqrt{\frac{1}{1!}b}, \sqrt{\frac{1}{2!}b^2}, \sqrt{\frac{1}{3!}b^3}, ..., \sqrt{\frac{1}{\infty!}b^{\infty}})
]
\end{align}

The next steps would involve expanding our first exponential function as a Taylor Series, finding the inner product and incorporating the result in our already found dot product. We will skip this steps for sake of demonstration purposes and to further simplify, we can now introduce a variable $s$ which equals the square root of the first term:

$$
s = \sqrt{e^{-\frac{1}{2}(a^2+b^2)}}
$$

We now can multiply our dot product with the new term $s$ and get our final dot product to quantify the similarities of our observations using the radial kernel in an infinite number of dimensions:

\begin{align}
K(a,b) &= e^{-\frac{1}{2}(a-b)^2}\\
&= 
(s, s\sqrt{\frac{1}{1!}a}, s\sqrt{\frac{1}{2!}a^2}, s\sqrt{\frac{1}{3!}a^3}, ..., s\sqrt{\frac{1}{\infty!}a^{\infty}})
\cdot
(s, s\sqrt{\frac{1}{1!}b}, s\sqrt{\frac{1}{2!}b^2}, s\sqrt{\frac{1}{3!}b^3}, ..., s\sqrt{\frac{1}{\infty!}b^{\infty}})
\end{align}

To simplify on what's happening here, let us show the theoratical transformation with the radial kernen into the third dimension:

In [ ]:
r = np.exp(-(np.sqrt(1/2 * X ** 2)).sum(axis = 1))

fig = plt.figure(figsize = (10, 8))
fig.set_facecolor('#EAEAF2')

ax = fig.add_subplot(projection = '3d')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('r')
ax.scatter3D(X[:, 0], X[:, 1], r, c = y, s = 40)
ax.view_init(elev = 10, azim = 45);

We can now use the radial kernel to train a model to separate the clusters of data points.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(14, 4))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)

for axi, C in zip(ax, [0.5, 100]):
    model = SVC(kernel = 'rbf', C = C).fit(X, y)
    axi.scatter(X[:, 0], X[:, 1], c = y, s = 40)
    plot_svc_decision_function(model, axi)
    axi.set_title('C = {0:.1f}'.format(C), size = 14)

## Kernels in scikit-learn

There are a vast number of kernels that can be used for the Support Vector Machine to find the decision boundaries on non-linear separatable data.

With _scikit-learn_ it is possible to use following kernels:
* Linear Kernel
* Polynomial Kernel
* Radial (Gaussian RBF) Kernel
* Hyperbolic Tangent (Sigmoid) Kernel
* _Precomputed_ Kernel

If you choose the _precomputed_ option, you are able to use any (self implemented) kernel for the Support Vector Machine. But keep in mind that you have to transform your _X\_train_ values with the kernel function before fitting the model.


## References

* Alashwal, H., Safaai Deris, and Razib M. Othman. “A Bayesian Kernel for the Prediction of Protein – Protein Interactions.” International Journal of Computational Intelligence 5, no. 2 (2009): 119-124.

* Basak, Jayanta. “A least square kernel machine with box constraints.” International Conference on Pattern Recognition 2008 1 (2008): 1-4.

* Boughorbel, S., Jean-Philippe Tarel, and Nozha Boujemaa. “Project-Imedia: Object Recognition.” INRIA – INRIA Activity Reports – RalyX.

* Fomel, Sergey. “Inverse B-spline interpolation.” Stanford Exploration Project, 2000.

* Genton, Marc G. “Classes of Kernels for Machine Learning: A Statistics Perspective.” Journal of Machine Learning Research 2 (2001) 299-312.

* Gunn, S. R. (1998, May). “Support vector machines for classification and regression.” Technical report, Faculty of Engineering, Science and Mathematics School of Electronics and Computer Science.

* Hamers B. “Kernel Models for Large Scale Applications”, Ph.D. , Katholieke Universiteit Leuven, Belgium, 2004.

* Hastie T., TIbshirani R., Friedman J. "The elements of statistical learning : data mining, inference, and prediction". New York, NY: Springer (2009), 417-440.

* Hichem Sahbi and François Fleuret. “Kernel methods and scale invariance using the triangular kernel”. INRIA Research Report, N-5143, March 2004.

* Hofmann, T., B. Schölkopf, and A. J. Smola. “Kernel methods in machine learning.” Ann. Statist. Volume 36, Number 3 (2008), 1171-1220.

* Howley, T. and Madden, M.G. “The genetic kernel support vector machine: Description and evaluation“. Artificial Intelligence Review. Volume 24, Number 3 (2005), 379-395.

* Hsuan-Tien Lin and Chih-Jen Lin. “A study on sigmoid kernels for SVM and the training of non-PSD kernels by SMO-type methods.” Technical report, Department of Computer Science, National Taiwan University, 2003.

* Huang, Lingkang. “Variable Selection in Multi-class Support Vector Machine and Applications in Genomic Data Analysis.” PhD Thesis, 2008.

* James G., Witten D., Hastie T., Tibshirani R. "An Introduction to Statistical Learning with Applications in R". New York, NY: Springer (2003), 337-372.

* Karatzoglou, A., Smola, A., Hornik, K. and Zeileis, A. “Kernlab – an R package for kernel Learning.”  (2004).

* Karatzoglou, A., Smola, A., Hornik, K. and Zeileis, A. “Kernlab – an S4 package for kernel methods in R.” J. Statistical Software, 11, 9 (2004).

* Karatzoglou, A., Smola, A., Hornik, K. and Zeileis, A. “R: Kernel Functions.” Documentation for package ‘kernlab’ version 0.9-5.

* Li Zhang, Weida Zhou, Licheng Jiao. Wavelet Support Vector Machine. IEEE Transactions on System, Man, and Cybernetics, Part B, 2004, 34(1): 34-39.

* Manning, Christopher D., Prabhakar Raghavan, and Hinrich Schütze. “Nonlinear SVMs.” The Stanford NLP (Natural Language Processing) Group.

* Micchelli, Charles. Interpolation of scattered data: Distance matrices and conditionally positive definite functions. Constructive Approximation 2, no. 1 (1986): 11-22.

* On-Line Prediction Wiki Contributors. “Kernel Methods.” On-Line Prediction Wiki.

* Sabri Boughorbel, Jean-Philippe Tarel, and Nozha Boujemaa. “Generalized histogram intersection kernel for image recognition”. Proceedings of the 2005 Conference on Image Processing, volume 3, pages 161-164, 2005.

* Shawkat Ali and Kate A. Smith. “Kernel Width Selection for SVM Classification: A Meta-Learning Approach.” International Journal of Data Warehousing & Mining, 1(4), 78-97, October-December 2005.

* Vedaldi, A. and Zisserman, A. Efficient Additive Kernels via Explicit Feature Maps. IEEE Transactions on Pattern Recognition and Machine Intelligence, Vol. XX, No. XX, June, 2011.

* Weisstein, Eric W. “Positive Semidefinite Matrix.” From MathWorld–A Wolfram.

* Wikipedia contributors, “Kernel methods,” Wikipedia, The Free Encyclopedia.

* Wikipedia contributors, “Kernel trick,” Wikipedia, The Free Encyclopedia.